# Fit-Predict Workflow

A demonstration of how it should like.

## Preliminaries

Imports and definitions

In [2]:
import pandas as pd
import prefect
import PxW 

from prefect import Task, Flow
from prefect.tasks.shell import ShellTask

## Fit

In [12]:
weka_cmd = """java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar"""

tree = "weka.classifiers.trees.J48"
tree_cmd = """{} -C 0.25 -M 2""".format(tree)

model_fn = "tree.model"
model_cmd = """-d {}""".format(model_fn)

data_fn = "../data/raw/iris.arff"
data_cmd = "-t {}".format(data_fn)

cmd = """{} {} {} -no-cv {}""".format(weka_cmd, tree_cmd, data_cmd, model_cmd)

shell = ShellTask()

with Flow("fit") as f:
    fit = shell(command=cmd)

status = f.run()

[2019-08-26 13:42:32,987] INFO - prefect.FlowRunner | Beginning Flow run for 'fit'
[2019-08-26 13:42:32,988] INFO - prefect.FlowRunner | Starting flow run.
[2019-08-26 13:42:32,991] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-08-26 13:42:32,993] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-08-26 13:42:32,994] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-08-26 13:42:33,350] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-08-26 13:42:33,351] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


In [5]:
print(status.result[fit].result.decode("utf-8"))


Options: -C 0.25 -M 2 

=== Classifier model (full training set) ===

J48 pruned tree
------------------

petal width (cm) <= 0.6: 0.0 (50.0)
petal width (cm) > 0.6
|   petal width (cm) <= 1.7
|   |   petal length (cm) <= 4.9: 1.0 (48.0/1.0)
|   |   petal length (cm) > 4.9
|   |   |   petal width (cm) <= 1.5: 2.0 (3.0)
|   |   |   petal width (cm) > 1.5: 1.0 (3.0/1.0)
|   petal width (cm) > 1.7: 2.0 (46.0/1.0)

Number of Leaves  : 	5

Size of the tree : 	9


Time taken to build model: 0.33 seconds

Time taken to test model on training data: 0.01 seconds

=== Error on training data ===

Correctly Classified Instances         147               98      %
Incorrectly Classified Instances         3                2      %
Kappa statistic                          0.97  
Mean absolute error                      0.0233
Root mean squared error                  0.108 
Relative absolute error                  5.2482 %
Root relative squared error             22.9089 %
Total Number of Instances   

## Predict

In [6]:
weka_cmd = """java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar"""

tree = "weka.classifiers.trees.J48"
tree_cmd = """{}""".format(tree)

model_fn = "tree.model"
model_cmd = """-l {}""".format(model_fn)

data_fn = "../data/raw/iris.arff"
data_cmd = "-T {}".format(data_fn)

out_fn = "pred.csv"
out_cmd = ""

cmd = """{} {} {} {} -p 0 """.format(weka_cmd, tree_cmd, data_cmd, model_cmd)

shell = ShellTask()


# Execution
with Flow("predict") as f:
    predict = shell(command=cmd)

status = f.run()

[2019-08-26 11:04:02,594] INFO - prefect.FlowRunner | Beginning Flow run for 'predict'
[2019-08-26 11:04:02,595] INFO - prefect.FlowRunner | Starting flow run.
[2019-08-26 11:04:02,599] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-08-26 11:04:02,600] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-08-26 11:04:02,601] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-08-26 11:04:02,824] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-08-26 11:04:02,826] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


In [8]:
s = status.result[predict].result.decode("utf-8")
#print(s)

### To Dataframe

This output has to get into Python again

In [10]:
from io import StringIO

df = pd.read_csv(StringIO(s),
                 index_col=0,
                 header=1,
                 usecols=[0,2],
                 lineterminator='\n',
                 sep='\s+')

In [11]:
df.head()

,predicted
inst#,
1,1:0.0
2,1:0.0
3,1:0.0
4,1:0.0
5,1:0.0
